In [1]:
import pyprob
import numpy as np
import ot
import torch
import cProfile

from pyprob.dis import ModelDIS
from showerSim import invMass_ginkgo
from torch.utils.data import DataLoader
from pyprob.nn.dataset import OnlineDataset
from pyprob.util import InferenceEngine
from pyprob.util import to_tensor
from pyprob import Model
import math
from pyprob.distributions import Normal
from pyprob.distributions.delta import Delta


import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as mpl_cm
plt.ion()

import sklearn as skl
from sklearn.linear_model import LinearRegression

from geomloss import SamplesLoss
sinkhorn = SamplesLoss(loss="sinkhorn", p=1, blur=.05)
def sinkhorn_t(x,y):
    x = to_tensor(x)
    y = torch.stack(y)
    return sinkhorn(x,y)

def ot_dist(x,y):
    # x = to_tensor(x)
    # y = torch.stack(y)
    x = np.array(x)
    y = np.array(torch.stack(y))
    a = ot.unif(len(x))
    b = ot.unif(len(y))
    Mat = ot.dist(x, y, metric='euclidean')
    #Mat1 /= Mat1.max()
    distance = to_tensor(ot.emd2(a,b,Mat))
    return distance


device = "cpu"

from pyprob.util import set_device
set_device(device)

obs_leaves = to_tensor([[44.57652381, 26.16169856, 25.3945314 , 25.64598258],
                           [18.2146321 , 10.70465096, 10.43553391, 10.40449709],
                           [ 6.47106713,  4.0435395,  3.65545951,  3.48697568],
                           [ 8.43764314,  5.51040615,  4.60990593,  4.42270416],
                           [26.61664145, 16.55894826, 14.3357362 , 15.12215264],
                           [ 8.62925002,  3.37121204,  5.19699   ,  6.00480461],
                           [ 1.64291837,  0.74506775,  1.01003622,  1.05626017],
                           [ 0.75525072,  0.3051808 ,  0.45721085,  0.51760643],
                           [39.5749915 , 18.39638928, 24.24717939, 25.29349408],
                           [ 4.18355659,  2.11145474,  2.82071304,  2.25221316],
                           [ 0.82932922,  0.29842766,  0.5799056 ,  0.509021  ],
                           [ 3.00825023,  1.36339397,  1.99203677,  1.79428211],
                           [ 7.20024308,  4.03280868,  3.82379277,  4.57441754],
                           [ 2.09953618,  1.28473579,  1.03554351,  1.29769683],
                           [12.21401828,  6.76059035,  6.94920042,  7.42823701],
                           [ 6.91438054,  3.68417135,  3.83782514,  4.41656731],
                           [ 1.97218904,  1.01632927,  1.08008339,  1.27454585],
                           [ 8.58164301,  5.06157833,  4.79691164,  4.99553141],
                           [ 5.97809522,  3.26557958,  3.4253764 ,  3.64894791],
                           [ 5.22842301,  2.94437891,  3.10292633,  3.00551074],
                           [15.40023764,  9.10884407,  8.93836964,  8.61970667],
                           [ 1.96101346,  1.24996337,  1.06923988,  1.06743143],
                           [19.81054106, 11.90268453, 11.60989346, 10.76953856],
                           [18.79470876, 11.429855  , 10.8377334 , 10.25112761],
                           [25.74331932, 15.63430056, 14.83860792, 14.07189108],
                           [ 9.98357576,  6.10090721,  5.68664128,  5.48748692],
                           [12.34604239,  7.78770185,  6.76075998,  6.78498685],
                           [21.24998531, 12.95180254, 11.9511704 , 11.87319933],
                           [ 7.80693733,  4.83117128,  4.27443559,  4.39602348],
                           [16.28983576,  9.66683929,  9.24891886,  9.28970032],
                           [ 2.50706736,  1.53153206,  1.36060018,  1.43002765],
                           [ 3.73938645,  2.06006639,  2.31013974,  2.09378969],
                           [20.2174725 , 11.88622367, 12.05106468, 11.05325362],
                           [ 9.48660008,  5.53665456,  5.54171966,  5.34966654],
                           [ 2.65812987,  1.64102742,  1.67392209,  1.25083707]])


QCD_mass = to_tensor(30.)
#rate=to_tensor([QCD_rate,QCD_rate]) #Entries: [root node, every other node] decaying rates. Choose same values for a QCD jet
jetdir = to_tensor([1.,1.,1.])
jetP = to_tensor(400.)
jetvec = jetP * jetdir / torch.linalg.norm(jetdir) ## Jetvec is 3-momentum. JetP is relativistic p.


# Actual parameters
pt_min = to_tensor(0.3**2)
M2start = to_tensor(QCD_mass**2)
jetM = torch.sqrt(M2start) ## Mass of initial jet
jet4vec = torch.cat((torch.sqrt(jetP**2 + jetM**2).reshape(-1), jetvec))
minLeaves = 1
maxLeaves = 10000 # unachievable, to prevent rejections
maxNTry = 100



class SimulatorModelDIS(invMass_ginkgo.SimulatorModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def dummy_bernoulli(self, jet):
        return True

    def forward(self, inputs=None):
        assert inputs is None # Modify code if this ever not met?
        # Sample parameter of interest from Unif(0,10) prior
        root_rate = pyprob.sample(pyprob.distributions.Uniform(0.01, 10.),
                                  name="decay_rate_parameter")
        decay_rate = pyprob.sample(pyprob.distributions.Uniform(0.01, 10.),
                                   name="decay_rate_parameter")
        # Simulator code needs two decay rates for (1) root note (2) all others
        # For now both are set to the same value
        inputs = [root_rate, decay_rate]
        jet = super().forward(inputs)
        return jet

# Make instance of the simulator
simulatorginkgo = SimulatorModelDIS(jet_p=jet4vec,  # parent particle 4-vector
                                    pt_cut=float(pt_min),  # minimum pT for resulting jet
                                    Delta_0= M2start,  # parent particle mass squared -> needs tensor
                                    M_hard=jetM,  # parent particle mass
                                    minLeaves=1,  # minimum number of jet constituents
                                    maxLeaves=10000,  # maximum number of jet constituents (a large value to stop expensive simulator runs)
                                    suppress_output=True,
                                    obs_leaves=obs_leaves)

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


In [2]:
outputs = simulatorginkgo.forward()

In [3]:
len(outputs['tree'])

23

In [4]:
len(outputs['leaves'])

12

In [5]:
outputs['tree']

tensor([[ 1., 16.],
        [ 2.,  9.],
        [ 3.,  6.],
        [ 4.,  5.],
        [-1., -1.],
        [-1., -1.],
        [ 7.,  8.],
        [-1., -1.],
        [-1., -1.],
        [10., 13.],
        [11., 12.],
        [-1., -1.],
        [-1., -1.],
        [14., 15.],
        [-1., -1.],
        [-1., -1.],
        [17., 20.],
        [18., 19.],
        [-1., -1.],
        [-1., -1.],
        [21., 22.],
        [-1., -1.],
        [-1., -1.]])

In [6]:
len(outputs['content'])

23

In [2]:
from showerSim import ginkgo_explicit

In [3]:
simulatorginkgo2 = ginkgo_explicit.SimulatorModel(jet_p=jet4vec,  # parent particle 4-vector
                                    control = 'all',
                                    pt_cut=float(pt_min),  # minimum pT for resulting jet
                                    Delta_0= M2start,  # parent particle mass squared -> needs tensor
                                    M_hard=jetM,  # parent particle mass
                                    minLeaves=1,  # minimum number of jet constituents
                                    maxLeaves=20,  # maximum number of jet constituents (a large value to stop expensive simulator runs)
                                    suppress_output=True,
                                    obs_leaves=obs_leaves,
                                    maxNTry=100)

In [4]:
root_rate = pyprob.sample(pyprob.distributions.Uniform(0.001, 10.),
                                  name="decay_rate_parameter")
decay_rate = pyprob.sample(pyprob.distributions.Uniform(0.001, 10.),
                                   name="decay_rate_parameter")

other_rates = [pyprob.sample(pyprob.distributions.Uniform(0.001,1)) for _ in range(12)]

all_rates = [9 , 9]+other_rates


In [5]:
out1 = simulatorginkgo2(all_rates)
out2 = simulatorginkgo2(all_rates)

25
20
26
20
21
21
25
22
23
20
22
21
23
26
22
21
24
20
21
26
24
26
20
26
22
25
22
21
26
30
25
23
25
22
20
22
25
23
21
21
23
25
23


In [20]:
out1['content']

tensor([[401.1234, 230.9401, 230.9401, 230.9401],
        [358.9539, 209.9012, 202.7195, 208.5414],
        [295.5461, 174.8354, 165.7785, 171.1228],
        [ 87.2712,  51.5128,  48.8183,  50.7830],
        [ 16.7639,   9.8412,   9.2146,   9.9628],
        [ 70.5073,  41.6716,  39.6038,  40.8202],
        [208.2749, 123.3226, 116.9602, 120.3398],
        [119.3909,  70.3295,  68.1389,  68.2974],
        [ 65.8038,  38.6919,  37.8270,  37.4457],
        [ 53.5871,  31.6376,  30.3119,  30.8517],
        [ 88.8840,  52.9931,  48.8213,  52.0424],
        [ 54.1379,  32.3045,  29.8428,  31.5710],
        [ 34.7461,  20.6886,  18.9785,  20.4714],
        [ 63.4079,  35.0658,  36.9410,  37.4187],
        [ 57.3849,  31.0779,  33.5034,  34.5410],
        [ 26.3802,  14.3894,  14.6887,  16.4507],
        [ 23.5249,  13.1653,  12.9245,  14.5882],
        [ 15.7195,   8.8267,   8.4768,   9.8641],
        [  7.8054,   4.3386,   4.4478,   4.7241],
        [  2.8553,   1.2240,   1.7642,   1.8625],


In [21]:
out2['content']

tensor([[ 4.0112e+02,  2.3094e+02,  2.3094e+02,  2.3094e+02],
        [ 3.5895e+02,  2.0990e+02,  2.0272e+02,  2.0854e+02],
        [ 2.9555e+02,  1.7484e+02,  1.6578e+02,  1.7112e+02],
        [ 8.7271e+01,  5.1513e+01,  4.8818e+01,  5.0783e+01],
        [ 4.4084e+01,  2.5742e+01,  2.4832e+01,  2.5770e+01],
        [ 4.3187e+01,  2.5771e+01,  2.3986e+01,  2.5013e+01],
        [ 2.0827e+02,  1.2332e+02,  1.1696e+02,  1.2034e+02],
        [ 1.0447e+02,  6.1193e+01,  5.9448e+01,  6.0285e+01],
        [ 1.8581e+01,  1.0650e+01,  1.0800e+01,  1.0729e+01],
        [ 8.5890e+01,  5.0543e+01,  4.8648e+01,  4.9556e+01],
        [ 1.0380e+02,  6.2129e+01,  5.7512e+01,  6.0054e+01],
        [ 2.4386e+01,  1.4645e+01,  1.3206e+01,  1.4346e+01],
        [ 7.9417e+01,  4.7485e+01,  4.4306e+01,  4.5708e+01],
        [ 6.3408e+01,  3.5066e+01,  3.6941e+01,  3.7419e+01],
        [ 3.7783e+01,  2.2211e+01,  2.0376e+01,  2.2713e+01],
        [ 2.1354e+01,  1.3104e+01,  1.1596e+01,  1.2237e+01],
        

In [22]:
out2['content']

tensor([[ 4.0112e+02,  2.3094e+02,  2.3094e+02,  2.3094e+02],
        [ 3.5895e+02,  2.0990e+02,  2.0272e+02,  2.0854e+02],
        [ 2.9555e+02,  1.7484e+02,  1.6578e+02,  1.7112e+02],
        [ 8.7271e+01,  5.1513e+01,  4.8818e+01,  5.0783e+01],
        [ 4.4084e+01,  2.5742e+01,  2.4832e+01,  2.5770e+01],
        [ 4.3187e+01,  2.5771e+01,  2.3986e+01,  2.5013e+01],
        [ 2.0827e+02,  1.2332e+02,  1.1696e+02,  1.2034e+02],
        [ 1.0447e+02,  6.1193e+01,  5.9448e+01,  6.0285e+01],
        [ 1.8581e+01,  1.0650e+01,  1.0800e+01,  1.0729e+01],
        [ 8.5890e+01,  5.0543e+01,  4.8648e+01,  4.9556e+01],
        [ 1.0380e+02,  6.2129e+01,  5.7512e+01,  6.0054e+01],
        [ 2.4386e+01,  1.4645e+01,  1.3206e+01,  1.4346e+01],
        [ 7.9417e+01,  4.7485e+01,  4.4306e+01,  4.5708e+01],
        [ 6.3408e+01,  3.5066e+01,  3.6941e+01,  3.7419e+01],
        [ 3.7783e+01,  2.2211e+01,  2.0376e+01,  2.2713e+01],
        [ 2.1354e+01,  1.3104e+01,  1.1596e+01,  1.2237e+01],
        

In [14]:
out2['tree'][1][0]

tensor(2.)

## BFS

In [2]:
from showerSim import ginkgo_breadthTrav

simulatorginkgo3 = ginkgo_breadthTrav.SimulatorModel(jet_p=jet4vec,  # parent particle 4-vector
                                    control = 'all',
                                    pt_cut=float(pt_min),  # minimum pT for resulting jet
                                    Delta_0= M2start,  # parent particle mass squared -> needs tensor
                                    M_hard=jetM,  # parent particle mass
                                    minLeaves=1,  # minimum number of jet constituents
                                    maxLeaves=50,  # maximum number of jet constituents (a large value to stop expensive simulator runs)
                                    suppress_output=True,
                                    obs_leaves=obs_leaves,
                                    maxNTry=100)

In [3]:
root_rate = pyprob.sample(pyprob.distributions.Uniform(0.001, 10.),
                                  name="decay_rate_parameter")
decay_rate = pyprob.sample(pyprob.distributions.Uniform(0.001, 10.),
                                   name="decay_rate_parameter")

other_rates = [pyprob.sample(pyprob.distributions.Uniform(0.001,1)) for _ in range(12)]

all_rates = [9 , 9]+other_rates

In [4]:
out1 = simulatorginkgo3(all_rates)
out2 = simulatorginkgo3(all_rates)

In [6]:
out1['leaves']

tensor([[ 3.9278e+00,  2.1584e+00,  2.3836e+00,  2.2523e+00],
        [ 6.8623e+00,  3.9617e+00,  3.8494e+00,  4.0700e+00],
        [ 3.2240e+01,  1.9625e+01,  1.6029e+01,  1.9931e+01],
        [ 4.9840e+00,  2.8449e+00,  3.6420e+00,  1.8637e+00],
        [ 1.1659e+01,  6.0431e+00,  6.0701e+00,  7.9068e+00],
        [ 2.9644e+00,  1.7380e+00,  1.3284e+00,  1.9950e+00],
        [ 2.6888e-01,  2.0891e-01,  1.4293e-01,  2.1669e-02],
        [ 3.4529e+00,  2.2334e+00,  2.3354e+00,  1.2094e+00],
        [ 7.4553e+00,  4.4857e+00,  3.8735e+00,  4.5177e+00],
        [ 7.9990e+01,  4.6615e+01,  4.4923e+01,  4.6983e+01],
        [ 2.1999e+01,  1.2228e+01,  1.3568e+01,  1.2260e+01],
        [ 4.0176e+00,  2.3317e+00,  2.4602e+00,  2.1565e+00],
        [ 2.5565e+01,  1.4949e+01,  1.6470e+01,  1.2604e+01],
        [ 2.0925e+01,  1.1977e+01,  1.1976e+01,  1.2284e+01],
        [ 1.1685e+01,  6.4449e+00,  6.4908e+00,  7.2665e+00],
        [ 3.7617e+00,  1.8279e+00,  2.9082e+00,  1.5226e+00],
        